In [1]:
#!/usr/bin/env python
import io
import os
import glob
import gzip
import pandas as pd
import json
#import matplotlib.pyplot as plt
#import seaborn as sns

In [2]:
directory_path = "/work/project/ext_016/RNA-Seq-Variant-Calling_1/source_dir_6/filtered/csv_files/*.csv"
# Get a list of all files that match the pattern
file_list = glob.glob(directory_path)
# Extract unique filenames without extensions
unique_file_list = list(set([os.path.basename(file_path).split(".")[0] for file_path in file_list]))
# Print the unique file list
print(len(unique_file_list))

26


In [5]:
# Get the metadata to have the information whether the sample is healthy or not
metadata = pd.read_csv("/work/project/ext_016/RNA-Seq-Variant-Calling_1/source_dir_6/filtered/srainfo/SraRunTable.csv")
pd.set_option('display.max_columns', None)
metadata

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,cell_type,Center Name,Consent,DATASTORE filetype,DATASTORE provider,DATASTORE region,disease,Experiment,GEO_Accession (exp),Instrument,LibraryLayout,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,create_date,version,Sample Name,source_name,SRA Study
0,SRR8181357,RNA-Seq,302,13106947074,PRJNA505280,SAMN10414139,8156799663,Peripheral blood mononuclear cells (PBMCs),GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",systemic lupus erythematosus,SRX5001351,GSM3466767,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2018-11-15T00:00:00Z,2018-11-13T11:03:00Z,1,GSM3466767,Peripheral blood mononuclear cells (PBMCs),SRP168421
1,SRR8181358,RNA-Seq,302,13793213384,PRJNA505280,SAMN10414138,8608719128,Peripheral blood mononuclear cells (PBMCs),GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",systemic lupus erythematosus,SRX5001352,GSM3466768,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2018-11-15T00:00:00Z,2018-11-13T11:07:00Z,1,GSM3466768,Peripheral blood mononuclear cells (PBMCs),SRP168421
2,SRR8181359,RNA-Seq,302,12034193546,PRJNA505280,SAMN10414137,7485804280,Peripheral blood mononuclear cells (PBMCs),GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",systemic lupus erythematosus,SRX5001353,GSM3466769,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2018-11-15T00:00:00Z,2018-11-13T11:06:00Z,1,GSM3466769,Peripheral blood mononuclear cells (PBMCs),SRP168421
3,SRR8181360,RNA-Seq,302,13061201926,PRJNA505280,SAMN10414136,8131915180,Peripheral blood mononuclear cells (PBMCs),GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",systemic lupus erythematosus,SRX5001354,GSM3466770,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2018-11-15T00:00:00Z,2018-11-13T11:07:00Z,1,GSM3466770,Peripheral blood mononuclear cells (PBMCs),SRP168421
4,SRR8181361,RNA-Seq,302,13186016110,PRJNA505280,SAMN10414135,8418114761,Peripheral blood mononuclear cells (PBMCs),GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",systemic lupus erythematosus,SRX5001355,GSM3466771,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2018-11-15T00:00:00Z,2018-11-13T11:06:00Z,1,GSM3466771,Peripheral blood mononuclear cells (PBMCs),SRP168421
5,SRR8181362,RNA-Seq,302,13337457332,PRJNA505280,SAMN10414134,8310726004,Peripheral blood mononuclear cells (PBMCs),GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",systemic lupus erythematosus,SRX5001356,GSM3466772,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2018-11-15T00:00:00Z,2018-11-13T11:07:00Z,1,GSM3466772,Peripheral blood mononuclear cells (PBMCs),SRP168421
6,SRR8181363,RNA-Seq,302,13754658554,PRJNA505280,SAMN10414133,8414579162,Peripheral blood mononuclear cells (PBMCs),GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",systemic lupus erythematosus,SRX5001357,GSM3466773,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2018-11-15T00:00:00Z,2018-11-13T11:08:00Z,1,GSM3466773,Peripheral blood mononuclear cells (PBMCs),SRP168421
7,SRR8181364,RNA-Seq,302,12243144628,PRJNA505280,SAMN10414132,7819543782,Peripheral blood mononuclear cells (PBMCs),GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",systemic lupus erythematosus,SRX5001358,GSM3466774,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2018-11-15T00:00:00Z,2018-11-13T11:05:00Z,1,GSM3466774,Peripheral blood mononuclear cells (PBMCs),SRP168421
8,SRR8181365,RNA-Seq,302,12298088092,PRJNA505280,SAMN10414131,7649015454,Peripheral blood mononuclear cells (PBMCs),GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",systemic lupus erythematosus,SRX5001359,GSM3466775,Illumina HiSeq 2500

In [4]:
# Dataset to merge the csv files only at the unc93b1 location
df = pd.DataFrame(columns=['POS','Dataset', 'Disease', 'Run', 'REF', 'ALT'])

In [6]:
# More efficient version avoiding row-by-row iteration
results=[]
for unique_file in unique_file_list:
    # Read and filter data
    temp = pd.read_csv(f"/work/project/ext_016/RNA-Seq-Variant-Calling_1/source_dir_6/filtered/csv_files/{unique_file}.csv")
    temp_confidence = pd.read_csv(f"/work/project/ext_016/RNA-Seq-Variant-Calling_1/source_dir_6/filtered/Confidence/{unique_file}_confidence.csv")
    
    #Filter positions
    filtered_temp = temp[(temp["POS"] >= 67990100) & (temp["POS"] <= 68005097)]
    
    # Merge dataframes
    merged_df = temp_confidence.merge(filtered_temp[['POS', 'REF','ALT']], 
                                  on='POS',
                                  how='left')
    
    # Get disease status
    disease_status = metadata.loc[metadata["Run"] == unique_file, "disease"].iloc[0]
    
    # Add constant columns
    merged_df['Dataset'] = 3
    merged_df['Run'] = unique_file
    merged_df['Disease'] = disease_status
    
    # Append to results
    results.append(merged_df[['Dataset', 'Run', 'Disease', 'POS', 'REF', 'ALT', 'GQ', 'QUAL', 'DP','gq_conf','qual_conf','dp_conf','confidence', 'FILTER']])

# Combine all results at once
df = pd.concat(results, ignore_index=True)
df.loc[df["Disease"] == "systemic lupus erythematosus", "Disease"] = "SLE"
df.loc[df["Disease"] == "healthy", "Disease"] = "Healthy"
df

,Dataset,Run,Disease,POS,REF,ALT,GQ,QUAL,DP,gq_conf,qual_conf,dp_conf,confidence,FILTER
0,3,SRR8181362,SLE,67990989,C,"T,<NON_REF>",28,20.64,7,0.998415,0.991370,0.054167,0.806747,PASS
1,3,SRR8181362,SLE,67991388,A,"C,<NON_REF>",99,0.00,990,1.000000,0.000000,1.000000,0.600000,PASS
2,3,SRR8181362,SLE,67991409,G,"T,<NON_REF>",99,0.00,1170,1.000000,0.000000,1.000000,0.600000,PASS
3,3,SRR8181362,SLE,67991412,C,"T,<NON_REF>",99,0.00,1197,1.000000,0.000000,1.000000,0.600000,PASS
4,3,SRR8181362,SLE,67991413,G,"A,<NON_REF>",99,0.00,1200,1.000000,0.000000,1.000000,0.600000,PASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180,3,SRR8181364,SLE,67998794,G,"A,<NON_REF>",6,37.31,2,0.748811,0.999814,0.018707,0.703192,PASS
1181,3,SRR8181364,SLE,67999893,G,"A,<NON_REF>",60,52.64,5,0.999999,0.999990,0.035571,0.807110,PASS
1182,3,SRR8181364,SLE,68002013,C,"T,<NON_REF>",6,59.31,2,0.748811,0.999990,0.018707,0.703262,PASS
1183,3,SRR8181364,SLE,68003158,G,"A,<NON_REF>",99,0.00,393,1.000000,0.000000,1.000000,0.600000,PASS


In [7]:
# Group by POS and calculate mean confidence
avg_confidence = df.groupby('POS')['confidence'].mean().reset_index()

# Merge back with original dataframe
df = df.merge(avg_confidence, 
              on='POS', 
              how='left',
              suffixes=('_old', ''))  # The original confidence column will get '_old' suffix

# Drop the old confidence column if it exists
if 'confidence_old' in df.columns:
    df = df.drop('confidence_old', axis=1)

print(df.head())

   Dataset         Run Disease       POS REF          ALT  GQ   QUAL    DP  \
0        3  SRR8181362     SLE  67990989   C  T,<NON_REF>  28  20.64     7   
1        3  SRR8181362     SLE  67991388   A  C,<NON_REF>  99   0.00   990   
2        3  SRR8181362     SLE  67991409   G  T,<NON_REF>  99   0.00  1170   
3        3  SRR8181362     SLE  67991412   C  T,<NON_REF>  99   0.00  1197   
4        3  SRR8181362     SLE  67991413   G  A,<NON_REF>  99   0.00  1200   

    gq_conf  qual_conf   dp_conf FILTER  confidence  
0  0.998415    0.99137  0.054167   PASS    0.806747  
1  1.000000    0.00000  1.000000   PASS    0.600000  
2  1.000000    0.00000  1.000000   PASS    0.600000  
3  1.000000    0.00000  1.000000   PASS    0.600000  
4  1.000000    0.00000  1.000000   PASS    0.600000  


In [8]:
# Save the final DataFrame to a CSV file
df.to_csv('/work/project/ext_016/RNA-Seq-Variant-Calling_1/dataset_3_positions_confidence.csv', index=False)